<a href="https://colab.research.google.com/github/dmachlanski/ncrm-causality-2021/blob/main/Day_2_CI_in_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install econml

     |████████████████████████████████| 3.0MB 5.2MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 358kB 42.5MB/s 
     |████████████████████████████████| 133kB 43.0MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp37-cp37m-linux_x86_64.whl size=490498 sha256=d06600886f41975ddc6215a87ac9f7566cd99cf038246270614c9f8a6f302b40
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap
  Found existing installation: pydot 1.3.0
    Uninstalling pydot-1.3.0:
      Successfully uninstalled pydot-1.3.0


In [ ]:
from econml.data.dgps import ihdp_surface_B
from econml.metalearners import XLearner
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
Y, T, X, expected_te = ihdp_surface_B()

y_train, y_test, t_train, t_test, x_train, x_test, te_train, te_test = train_test_split(Y, T, X, expected_te, test_size=0.2, random_state=1, stratify=T)

In [ ]:
# X-Learner
xl = XLearner(models=RandomForestRegressor(), propensity_model=RandomForestClassifier())
xl.fit(y_train, t_train, X=x_train)

xl_te_in = xl.effect(x_train)
xl_te_out = xl.effect(x_test)

In [ ]:
t_train_2d = t_train.reshape(-1, 1)
t_test_2d = t_test.reshape(-1, 1)

# Random Forest
rf = RandomForestRegressor()
rf.fit(np.concatenate([x_train, t_train_2d], axis=1), y_train)

rf_y0_in = rf.predict(np.concatenate([x_train, np.zeros_like(t_train_2d)], axis=1))
rf_y1_in = rf.predict(np.concatenate([x_train, np.ones_like(t_train_2d)], axis=1))

rf_y0_out = rf.predict(np.concatenate([x_test, np.zeros_like(t_test_2d)], axis=1))
rf_y1_out = rf.predict(np.concatenate([x_test, np.ones_like(t_test_2d)], axis=1))

rf_te_in = rf_y1_in - rf_y0_in
rf_te_out = rf_y1_out - rf_y0_out

In [ ]:
def rmse(a, b):
    return np.sqrt(((a - b)**2).mean())

def ate_error(pred_te, true_te):
  return np.abs(np.mean(pred_te) - np.mean(true_te))

def pehe_error(pred_te, true_te):
  return rmse(true_te, pred_te)

In [ ]:
xl_ate_in, xl_pehe_in = ate_error(xl_te_in, te_train), pehe_error(xl_te_in, te_train)
xl_ate_out, xl_pehe_out = ate_error(xl_te_out, te_test), pehe_error(xl_te_out, te_test)

rf_ate_in, rf_pehe_in = ate_error(rf_te_in, te_train), pehe_error(rf_te_in, te_train)
rf_ate_out, rf_pehe_out = ate_error(rf_te_out, te_test), pehe_error(rf_te_out, te_test)

In [ ]:
results = []
results.append(['XL', xl_ate_in, xl_pehe_in, xl_ate_out, xl_pehe_out])
results.append(['RF', rf_ate_in, rf_pehe_in, rf_ate_out, rf_pehe_out])

cols = ['Method', 'ATE in', 'PEHE in', 'ATE out', 'PEHE out']

df = pd.DataFrame(results, columns=cols)
df

,Method,ATE in,PEHE in,ATE out,PEHE out
0,XL,0.228419,2.491936,0.110626,2.286824
1,RF,0.060529,1.376601,0.114791,1.751148
